# Run this notebook 
include complete calendar and public holidays

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f
import holidays



## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
inbound_2022_dry = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-11-01')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    # 'max_depth':2,      
}
# variables 
feature_col = ['x_7days', 'x_14days', 'x_21days']
pred_period=6
end_horizon = 7
fc_freq=7
training_period= 60

### Clean the data

removes all sundays and holidays in the complete dataset

In [0]:
shift_periods = [shift_period_1, shift_period_2, shift_period_3]
column_names = ['x_7days', 'x_14days', 'x_21days']

for i, period in enumerate(shift_periods):
    inbound_2022_dry[column_names[i]] = inbound_2022_dry['Dry Actuals'].shift(period)


In [0]:
inbound_2022_dry

,Dates,Dry Actuals,x_7days,x_14days,x_21days
0,2022-01-03,84224.0,NaN,NaN,NaN
1,2022-01-04,49564.0,NaN,NaN,NaN
2,2022-01-05,45890.0,NaN,NaN,NaN
3,2022-01-06,26976.0,NaN,NaN,NaN
4,2022-01-07,37044.0,NaN,NaN,NaN
...,...,...,...,...,...
358,2022-12-27,63764.0,75921.0,68514.0,76287.0
359,2022-12-28,47680.0,58697.0,65979.0,63301.0
360,2022-12-29,39552.0,62334.0,56286.0,54256.0
361,2022-12-30,61119.0,55510.0,50336.0,52050.0


In [0]:
#drop Nan and sundays and holidays with Zero in Actuals 
inbound_2022_dry = inbound_2022_dry.dropna().loc[inbound_2022_dry['Dry Actuals'] != 0].reset_index(drop=True)


In [0]:
inbound_2022_dry

,Dates,Dry Actuals,x_7days,x_14days,x_21days
0,2022-01-24,102266.0,81534.0,71540.0,84224.0
1,2022-01-25,80584.0,72886.0,63644.0,49564.0
2,2022-01-26,70974.0,53783.0,66433.0,45890.0
3,2022-01-27,67565.0,0.0,52802.0,26976.0
4,2022-01-28,57571.0,52025.0,43554.0,37044.0
...,...,...,...,...,...
275,2022-12-27,63764.0,75921.0,68514.0,76287.0
276,2022-12-28,47680.0,58697.0,65979.0,63301.0
277,2022-12-29,39552.0,62334.0,56286.0,54256.0
278,2022-12-30,61119.0,55510.0,50336.0,52050.0


### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_dry[['Dates', 'Dry Actuals']].copy()#original non filtered dataset, 


# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=pred_period) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  # end_train_date = backtest_start_date + dt.timedelta(days=training_period)
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  print(f"Backtesting training ending on: {end_train_date}") 

  lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{lgbm_train.shape} ")
  # Convert Dates column to datetime
  lgbm_train['Dates'] = pd.to_datetime(lgbm_train['Dates'])
    
  #Train features 
  train_features_df = inbound_2022_dry[(lgbm_train['Dates'] >= start_train_date) &(inbound_2022_dry['Dates'] < end_train_date)].copy()
  features_values_train=train_features_df.iloc[:, -3:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date+dt.timedelta(days=15), periods=pred_period, freq='D')

  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df

  prediction_dates = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration,predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=end_horizon)
 
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-03-04 00:00:00
print the shape:(32, 2) 
[LightGBM] [Warning] There are no meaningful features, as all feature values are constant.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score 67662.531250
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirem

In [0]:
backtest_results

,ds,yhat
45,2022-03-19,67662.531250
46,2022-03-21,67662.531250
47,2022-03-22,67662.531250
48,2022-03-23,67662.531250
49,2022-03-24,67662.531250
...,...,...
274,2022-12-26,88344.615512
275,2022-12-27,88344.615512
276,2022-12-28,65188.241500
277,2022-12-29,64191.798847


### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-03-19,48653.0,62014.766938,67662.531250
1,2022-03-21,99066.0,92036.016448,67662.531250
2,2022-03-22,73190.0,65446.552078,67662.531250
3,2022-03-23,55690.0,59679.316195,67662.531250
4,2022-03-24,53803.0,56573.711240,67662.531250
...,...,...,...,...
190,2022-12-26,90911.0,95488.000000,88344.615512
191,2022-12-27,63764.0,71459.270000,88344.615512
192,2022-12-28,47680.0,71916.320000,65188.241500
193,2022-12-29,39552.0,60346.346000,64191.798847


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i], fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-03-19,48653.0,62014.766938,67662.531250,13361.766938,19009.531250,13361.766938,19009.531250,1.785368e+08,3.613623e+08,0.274634,True,0.390717,True
1,2022-03-21,99066.0,92036.016448,67662.531250,-7029.983552,-31403.468750,7029.983552,31403.468750,4.942067e+07,9.861778e+08,0.070963,False,0.316995,True
2,2022-03-22,73190.0,65446.552078,67662.531250,-7743.447922,-5527.468750,7743.447922,5527.468750,5.996099e+07,3.055291e+07,0.105799,True,0.075522,False
3,2022-03-23,55690.0,59679.316195,67662.531250,3989.316195,11972.531250,3989.316195,11972.531250,1.591464e+07,1.433415e+08,0.071634,False,0.214985,True
4,2022-03-24,53803.0,56573.711240,67662.531250,2770.711240,13859.531250,2770.711240,13859.531250,7.676841e+06,1.920866e+08,0.051497,False,0.257598,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2022-12-26,90911.0,95488.000000,88344.615512,4577.000000,-2566.384488,4577.000000,2566.384488,2.094893e+07,6.586329e+06,0.050346,False,0.028230,False
191,2022-12-27,63764.0,71459.270000,88344.615512,7695.270000,24580.615512,7695.270000,24580.615512,5.921718e+07,6.042067e+08,0.120684,True,0.385494,True
192,2022-12-28,47680.0,71916.320000,65188.241500,24236.320000,17508.241500,24236.320000,17508.241500,5.873992e+08,3.065385e+08,0.508312,True,0.367203,True
193,2022-12-29,39552.0,60346.346000,64191.798847,20794.346000,24639.798847,20794.346000,24639.798847,4.324048e+08,6.071197e+08,0.525747,True,0.622972,True


In [0]:

days_out_range= backtest_df_merge.groupby('Dates').agg({'Outside_range_Dry_LGBM':'sum'}).reset_index()
days_out_range


,Dates,Outside_range_Dry_LGBM
0,2022-03-19,1
1,2022-03-21,1
2,2022-03-22,0
3,2022-03-23,1
4,2022-03-24,1
...,...,...
190,2022-12-26,0
191,2022-12-27,1
192,2022-12-28,1
193,2022-12-29,1


In [0]:
filter_df = days_out_range[days_out_range['Outside_range_Dry_LGBM'] != 0].reset_index(drop=True)
filter_df

,Dates,Outside_range_Dry_LGBM
0,2022-03-19,1
1,2022-03-21,1
2,2022-03-23,1
3,2022-03-24,1
4,2022-03-26,1
...,...,...
135,2022-12-24,1
136,2022-12-27,1
137,2022-12-28,1
138,2022-12-29,1


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()


In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,1.950000e+02,1.950000e+02,195.000000,195.000000,195,195
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116,140
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,67211.492308,69790.276681,67543.583997,2578.784373,332.091690,7153.711886,16158.953575,1.120397e+08,5.318794e+08,0.134142,0.254180,NaN,NaN
7,std,24813.392838,20760.728265,13656.014616,10292.368971,23119.477457,7821.629077,16497.374749,3.422036e+08,1.438871e+09,0.229922,0.294985,NaN,NaN
8,min,20444.000000,38016.300000,43852.860520,-59204.840000,-119741.182780,12.280000,55.209182,1.507984e+02,3.048054e+03,0.000173,0.000811,NaN,NaN
9,25%,50623.500000,56823.361362,54355.797953,-1827.375309,-8875.170161,2427.072000,5942.123256,5.892363e+06,3.531408e+07,0.033930,0.087481,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')